# Cinemática Braço Robtótico Leticya

### Bancada de Testes de Pré-Carga de Fusos de Esfera Integrada com Braço Robótico

### Referências
- https://www.youtube.com/watch?v=4WRhVqQaZTE
- https://robotics.stackexchange.com/questions/8516/getting-pitch-yaw-and-roll-from-rotation-matrix-in-dh-parameter
- https://automaticaddison.com/how-to-assign-denavit-hartenberg-frames-to-robotic-arms/
- https://d3cw3dd2w32x2b.cloudfront.net/wp-content/uploads/2012/07/euler-angles1.pdf

# 1.0 Bibliotecas e Funções

In [1]:
import mpmath
import numpy as np
import sympy as sym

def Rot_x(Rx):
    return sym.Matrix([
                      [1, 0,           0,            0],
                      [0, sym.cos(Rx), -sym.sin(Rx), 0],
                      [0, sym.sin(Rx), sym.cos(Rx),  0],
                      [0, 0,           0,            1]  
                      ])

def Rot_y(Ry):
    return sym.Matrix([
                      [sym.cos(Ry),  0, sym.sin(Ry), 0],
                      [0,            1, 0,           0],
                      [-sym.sin(Ry), 0, sym.cos(Ry), 0],
                      [0,            0, 0,           1]  
                      ])

def Rot_z(Rz):
    return sym.Matrix([
                      [sym.cos(Rz), -sym.sin(Rz), 0, 0],
                      [sym.sin(Rz), sym.cos(Rz),  0, 0],
                      [0,           0,            1, 0],
                      [0,           0,            0, 1]
                      ])

def Transl_x(Tx):
    return sym.Matrix([
                      [1, 0, 0, Tx],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]
                      ])

def Transl_y(Ty):
    return sym.Matrix([
                      [1, 0, 0, 0 ],
                      [0, 1, 0, Ty],
                      [0, 0, 1, 0 ],
                      [0, 0, 0, 1 ]
                      ])

def Transl_z(Tz):
    return sym.Matrix([
                      [1, 0, 0, 0 ],
                      [0, 1, 0, 0 ],
                      [0, 0, 1, Tz],
                      [0, 0, 0, 1 ] 
                      ])

# 2.0 Cinemática direta

### 2.1 Configuração do Robô

In [2]:
# Distâncias entre os eixos
d1 = 1
d2 = 1
d3 = 1
d4 = 1
d5 = 1
d6 = 1
d7 = 1

### 2.2 Simbologias para cálculo

In [3]:
# Variáveis de distância
sym_d1 = sym.Symbol('d1')
sym_d2 = sym.Symbol('d2')
sym_d3 = sym.Symbol('d3')
sym_d4 = sym.Symbol('d4')
sym_d5 = sym.Symbol('d5')
sym_d6 = sym.Symbol('d6')
sym_d7 = sym.Symbol('d7')

# Variáveis de offset
sym_off_Tx = sym.Symbol('OFFSET_Tx')
sym_off_Ty = sym.Symbol('OFFSET_Ty')
sym_off_Tz = sym.Symbol('OFFSET_Tz')
sym_off_Rx = sym.Symbol('OFFSET_Rx')
sym_off_Ry = sym.Symbol('OFFSET_Ry')
sym_off_Rz = sym.Symbol('OFFSET_Rz')

# Rotações atuais dos motores
sym_M1 = sym.Symbol('M1')
sym_M2 = sym.Symbol('M2')
sym_M3 = sym.Symbol('M3')
sym_M4 = sym.Symbol('M4')
sym_M5 = sym.Symbol('M5')
sym_M6 = sym.Symbol('M6')

# Eixos interpolados finais
sym_Tx = sym.Symbol('Tx')
sym_Ty = sym.Symbol('Ty')
sym_Tz = sym.Symbol('Tz')
sym_Rx = sym.Symbol('Rx')
sym_Ry = sym.Symbol('Ry')
sym_Rz = sym.Symbol('Rz')

### 2.3 Matrizes de rotação e translação homogêneas

In [4]:
# Transformações dos sistemas de coordenadas da base (Sistema Inercial) até o ponto de pivotamento (Garra)
T12 = Rot_z(sym_M1) * Transl_z(sym_d1)
T23 = Rot_x(sym_M2) * Transl_y(sym_d2)
T34 = Rot_x(sym_M3) * Transl_y(sym_d3)
T45 = Rot_y(sym_M4) * Transl_y(sym_d4)
T56 = Rot_x(sym_M5) * Transl_y(sym_d5)
T67 = Rot_z(sym_M6) * Transl_z(sym_d6) * Transl_y(sym_d7)

# Matriz final
T = T12*T23*T34*T45*T56*T67

### 2.4 Extração das equações da Cinemática Direta

In [7]:
# Cálculo da referência da ponta da Garra ao ponto Zero para extrair as translações
XYZ = T*sym.Matrix([[0], [0], [0], [1]])
X = sym.simplify(XYZ[0])
Y = sym.simplify(XYZ[1])
Z = sym.simplify(XYZ[2])

# Criação das equações simbólicas das translações
equations = [X-sym_Tx, Y-sym_Ty, Z-sym_Tz]

# Solução das equações de translação
Tx = sym.solve(equations[0], sym_Tx)[0]
Ty = sym.solve(equations[1], sym_Ty)[0]
Tz = sym.solve(equations[2], sym_Tz)[0]

# Declaração de alguns índices da Matriz T para encontrar as equações de rotação
T00 = T[0,0]
T01 = T[0,1]
T02 = T[0,2]
T10 = T[1,0]
T11 = T[1,1]
T12 = T[1,2]
T20 = T[2,0]
T21 = T[2,1]
T22 = T[2,2]





# Solução das equações de rotação
#Rx = sym.simplify(sym.atan2(T12, T22))
#Ry = sym.simplify(sym.atan2(-T02, sym.sqrt((T00**2)+(T01**2))))
#Rz = sym.simplify(sym.atan2(sym.sin(Rx)*T20-sym.cos(Rx)*T10, sym.cos(Rx)*T11-sym.sin(Rx)*T21))

# ADEQUAÇÃO DE SINAIS - TESTAR E APAGAR O DE CIMA
Rx = sym.simplify(-sym.atan2(T12, T22))
Ry = sym.simplify(-sym.atan2(-T02, sym.sqrt((T00**2)+(T01**2))))
Rz = sym.simplify(-sym.atan2(sym.sin(-Rx)*T20-sym.cos(-Rx)*T10, sym.cos(-Rx)*T11-sym.sin(-Rx)*T21))

### 2.5 Print da Cinemática Direta para o CLP (Linguagem ST)

In [36]:
print('Tx := ' + (str(Tx) + ';\n').upper().replace('D', 'd'))
print('Ty := ' + (str(Ty) + ';\n').upper().replace('D', 'd'))
print('Tz := ' + (str(Tz) + ';\n').upper().replace('D', 'd'))
print('Rx := ' + (str(Rx) + ';\n').upper().replace('D', 'd'))
print('Ry := ' + (str(Ry) + ';\n').upper().replace('D', 'd'))
print('Rz := ' + (str(Rz) + ';'  ).upper().replace('D', 'd'))

Tx := -d2*SIN(M1)*COS(M2) - d3*SIN(M1)*COS(M2 + M3) - d4*SIN(M1)*COS(M2 + M3) + d5*SIN(M1)*SIN(M5)*SIN(M2 + M3)*COS(M4) - d5*SIN(M1)*COS(M5)*COS(M2 + M3) + d5*SIN(M4)*SIN(M5)*COS(M1) + d6*SIN(M1)*SIN(M5)*COS(M2 + M3) + d6*SIN(M1)*SIN(M2 + M3)*COS(M4)*COS(M5) + d6*SIN(M4)*COS(M1)*COS(M5) + d7*SIN(M1)*SIN(M4)*SIN(M6)*SIN(M2 + M3) + d7*SIN(M1)*SIN(M5)*SIN(M2 + M3)*COS(M4)*COS(M6) - d7*SIN(M1)*COS(M5)*COS(M6)*COS(M2 + M3) + d7*SIN(M4)*SIN(M5)*COS(M1)*COS(M6) - d7*SIN(M6)*COS(M1)*COS(M4);

Ty := d2*COS(M1)*COS(M2) + d3*COS(M1)*COS(M2 + M3) + d4*COS(M1)*COS(M2 + M3) + d5*SIN(M1)*SIN(M4)*SIN(M5) - d5*SIN(M5)*SIN(M2 + M3)*COS(M1)*COS(M4) + d5*COS(M1)*COS(M5)*COS(M2 + M3) + d6*SIN(M1)*SIN(M4)*COS(M5) - d6*SIN(M5)*COS(M1)*COS(M2 + M3) - d6*SIN(M2 + M3)*COS(M1)*COS(M4)*COS(M5) + d7*SIN(M1)*SIN(M4)*SIN(M5)*COS(M6) - d7*SIN(M1)*SIN(M6)*COS(M4) - d7*SIN(M4)*SIN(M6)*SIN(M2 + M3)*COS(M1) - d7*SIN(M5)*SIN(M2 + M3)*COS(M1)*COS(M4)*COS(M6) + d7*COS(M1)*COS(M5)*COS(M6)*COS(M2 + M3);

Tz := d1 + d2*SIN(M2)

### 2.6 Simulação de Funcionamento da Cinemática Direta

In [37]:
# Posições simuladas dos motores (em graus)
m1 = 0
m2 = 0
m3 = 0
m4 = 0
m5 = 0
m6 = 0

# Conversão para radianos para uso na simulação
m1_rad = mpmath.radians(m1)
m2_rad = mpmath.radians(m2)
m3_rad = mpmath.radians(m3)
m4_rad = mpmath.radians(m4)
m5_rad = mpmath.radians(m5)
m6_rad = mpmath.radians(m6)

# Execução dos cálculos
forward_Tx = sym.N(Tx.subs([(sym_M1, m1_rad), (sym_M2, m2_rad), (sym_M3, m3_rad), (sym_M4, m4_rad), (sym_M5, m5_rad), (sym_M6, m6_rad), (sym_d1, d1), (sym_d2, d2), (sym_d3, d3), (sym_d4, d4), (sym_d5, d5), (sym_d6, d6), (sym_d7, d7)]))
forward_Ty = sym.N(Ty.subs([(sym_M1, m1_rad), (sym_M2, m2_rad), (sym_M3, m3_rad), (sym_M4, m4_rad), (sym_M5, m5_rad), (sym_M6, m6_rad), (sym_d1, d1), (sym_d2, d2), (sym_d3, d3), (sym_d4, d4), (sym_d5, d5), (sym_d6, d6), (sym_d7, d7)]))
forward_Tz = sym.N(Tz.subs([(sym_M1, m1_rad), (sym_M2, m2_rad), (sym_M3, m3_rad), (sym_M4, m4_rad), (sym_M5, m5_rad), (sym_M6, m6_rad), (sym_d1, d1), (sym_d2, d2), (sym_d3, d3), (sym_d4, d4), (sym_d5, d5), (sym_d6, d6), (sym_d7, d7)]))
forward_Rx = sym.N(Rx.subs([(sym_M1, m1_rad), (sym_M2, m2_rad), (sym_M3, m3_rad), (sym_M4, m4_rad), (sym_M5, m5_rad), (sym_M6, m6_rad), (sym_d1, d1), (sym_d2, d2), (sym_d3, d3), (sym_d4, d4), (sym_d5, d5), (sym_d6, d6), (sym_d7, d7)]))
forward_Ry = sym.N(Ry.subs([(sym_M1, m1_rad), (sym_M2, m2_rad), (sym_M3, m3_rad), (sym_M4, m4_rad), (sym_M5, m5_rad), (sym_M6, m6_rad), (sym_d1, d1), (sym_d2, d2), (sym_d3, d3), (sym_d4, d4), (sym_d5, d5), (sym_d6, d6), (sym_d7, d7)]))
forward_Rz = sym.N(Rz.subs([(sym_M1, m1_rad), (sym_M2, m2_rad), (sym_M3, m3_rad), (sym_M4, m4_rad), (sym_M5, m5_rad), (sym_M6, m6_rad), (sym_d1, d1), (sym_d2, d2), (sym_d3, d3), (sym_d4, d4), (sym_d5, d5), (sym_d6, d6), (sym_d7, d7)]))

# Print das translações calculadas
print('Tx: %.2f' % (forward_Tx))
print('Ty: %.2f' % (forward_Ty))
print('Tz: %.2f' % (forward_Tz))
print('Rx: %.2f' % (mpmath.degrees(forward_Rx)))
print('Ry: %.2f' % (mpmath.degrees(forward_Ry)))
print('Rz: %.2f' % (mpmath.degrees(forward_Rz)))

Tx: 0.00
Ty: 5.00
Tz: 2.00
Rx: 0.00
Ry: 0.00
Rz: 0.00


# 3.0 Cinemática Inversa

### 3.1 Matriz Jacobiana

In [38]:
equations = sym.Matrix([X, Y, Z, Rx, Ry, Rz])
motors = sym.Matrix([sym_M1, sym_M2, sym_M3, sym_M4, sym_M5, sym_M6])

In [68]:
J = equations.jacobian(motors)
J = J**-1

KeyboardInterrupt: 

In [ ]:
G = sym.Matrix([sym_Tx-X, sym_Ty-Y, sym_Tz-Z, sym_Rx-Rx, sym_Ry-Ry, sym_Rz-Rz])
e = np.array([[0],[0],[0],[0],[0],[0]])

In [ ]:
for i in range(5):
    J_evaluated = sym.N(J.subs([(sym_M1,e[0][0]), (sym_M2,e[1][0]), (sym_M3,e[2][0]), (sym_M4,e[3][0]), (sym_M5,e[4][0]), (sym_M6,e[5][0])]))
    G_evaluated = sym.N(G.subs([(sym_M1,e[0][0]), (sym_M2,e[1][0]), (sym_M3,e[2][0]), (sym_M4,e[3][0]), (sym_M5,e[4][0]), (sym_M6,e[5][0])]))
    e = np.array(e + J_evaluated*G_evaluated)
    print(i)

In [ ]:
for i in range(len(e)):
    print(e[i][0])
    print()